# Test models

This notebook displays $S(q, \omega)$ in linear and log scale for a selected model from the QENS library. 

This model can be convoluted with a Gaussian instrument profile. 

<a id='Table of Contents'></a><h3>Table of Contents</h3>
- <a href='#imports'>Load packages</a>  
- <a href='#general_settings'>Common settings</a>  
- <a href='#choice_model'>Choice of model</a>  
- <a href='#display_model'>Plot</a>


<div class="alert alert-block alert-danger">
Only Python 3 is supported.
</div>

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Load packages</h2>

In [ ]:
from __future__ import print_function

import sys
import numpy as np
import matplotlib.pyplot as plt

from inspect import signature

import QENSmodels

# for interactive plot
from IPython.display import Javascript, display

import panel
panel.extension()
import panel.widgets as pnw

(<a href='#Table of Contents'>Top</a>)<a id='general_settings'></a><h2>Common settings</h2>

In [ ]:
# Vector of Q and omega values
# omega array should be symmetric (w_min = -w_max) and have a uniform step
q = np.arange(0.25, 2.1, 0.25)
omega = np.arange(-2, 2.01, 0.01)

# Parameters
scale = 1.0
center = 0.0
background = 0.0

(<a href='#Table of Contents'>Top</a>)<a id='choice_model'></a><h2>Choice of model</h2>

In [ ]:
# The `definitions` dictionary is used when asking the user to enter the values of the selected model
definitions = {
    "scale": "Scale",
    "center": "Center (in ps)",
    "Nsites": ' Number of sites on circle (integer>2): ',
    "radius": 'Radius (in Angstrom): ',
    "resTime": 'Residence time between jumps (in ps): ',
    "D": 'Self-diffusion coefficient (in Angstrom^2/ps): ',
    "variance_ux": 'Variance of Gaussian random variable (in Angstrom^2 ): ',
    "DR": 'Rotational diffusion coefficient (in 1/ps)?',
    "L": 'Jump length (in Angstrom)',
    "sigma": 'Standard deviation of the Gaussian distribution: '
}


options_list=['BrownianTranslationalDiffusion', 
              'ChudleyElliotDiffusion', 
              'EquivalentSitesCircle', 
              'GaussianModel3D', 
              'IsotropicRotationalDiffusion', 
              'JumpSitesLogNormDist', 
              'JumpTranslationalDiffusion']

model = pnw.ToggleGroup(options=options_list, widget_type='box',behavior='radio')

widgets_models = panel.Row("#### Models:", model)

checkbox_res = pnw.Checkbox(name='Use Gaussian resolution', height=15, value=False)

checkbox_backgd = pnw.Checkbox(name='Use flat background', height=15, value=False)

options_model = panel.Column(widgets_models, checkbox_res, checkbox_backgd)

options_model

In [ ]:
if checkbox_res.value is True:
    sig = pnw.TextInput(
        placeholder='Sigma of Gaussian resolution',
        width=250,
        name='Sigma:')
    display(sig)
else:
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:
if checkbox_backgd.value is True:
    backgd = pnw.TextInput(
        placeholder='Value of flat background',
        width=250,
        name='Flat background:')
    display(backgd)
else:
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

(<a href='#Table of Contents'>Top</a>)<a id='display_model'></a><h2>Plot</h2>

In [ ]:
# Resolution    
if checkbox_res.value is True:
    sigma = float(sig.value)
    resol = np.exp(-omega**2/(2*sigma**2)) / (sigma*np.sqrt(2*np.pi))
else:
    resol = None

function_to_plot = eval("QENSmodels.sqw"+model.value)
sig = signature(function_to_plot)
params = sig.parameters

print('\033[1m' + "Selected model: "+ '\033[0m' + model.value)

list_arg=[]
for key in params.keys():
    if key != 'q' and key!= 'w':
        value = input(definitions[key] + "(default: " + str(params[key].default) + "): ")
        if value is "":
            value = params[key].default
        list_arg.append(float(value))

In [ ]:
model_sqw = function_to_plot(omega, q, *list_arg)

if resol is not None:
    sqw = np.asarray([np.convolve(model_sqw[i,:], resol/resol.sum(), mode='same') for i in range(len(q))])
else:
    sqw = model_sqw
    
# Background
if checkbox_backgd.value is True:
    sqw += float(backgd.value)

In [ ]:
# Plot
fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(121)
for i in range(q.size):
    integral = np.trapz(sqw[i,:], omega)
    print ("Integral S(Q=",q[i],",w) = ", integral)
    ax1.plot(omega, sqw[i,:])  
ax1.grid(True)
ax1.set_xlabel('Omega', fontsize=20)
ax1.set_ylabel('S(Q,w)', fontsize=20)
ax1.set_yscale('log')
ax1.tick_params(labelsize=16)

ax2 = fig.add_subplot(122)
for i in range(q.size):
    ax2.plot(omega, sqw[i,:], label="q={}".format(str(q[i])))
ax2.grid(True)
ax2.set_xlabel('Omega', fontsize=20)
ax2.set_ylabel('S(Q,w)', fontsize=20)
ax2.tick_params(labelsize=16)

plt.tight_layout()
plt.suptitle('S(Q, w) log and linear scales for {model}'.format(model=model.value),x=0.55,y=1.025, fontsize=20)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()   